# Interview task

In [218]:
MODEL_NAME = "Rocketknight1/falcon-rw-1b"
RANDOM_STATE = 3

## check GPU

In [219]:
import torch
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
print(f'device: {device}')

device: cuda:0


## Check dataset

In [220]:
import pandas as pd

In [221]:
df = pd.read_csv('substance_interactions.csv')

In [222]:
df.head()

,PREDICATION_ID,PMID,PREDICATE,INDICATOR_TYPE,PREDICATE_START_INDEX,PREDICATE_END_INDEX,SUBJECT_TEXT,SUBJECT_SEMTYPE,SUBJECT_START_INDEX,SUBJECT_END_INDEX,...,OBJECT_START_INDEX,OBJECT_END_INDEX,OBJECT_SCORE,OBJECT_DIST,OBJECT_MAXDIST,OBJECT_CUI,OBJECT_NOVELTY,TYPE,SENTENCE,LABEL
0,P3100,6499897,INTERACTS_WITH,NOM,1298,1304,SA,orch,1235,1237,...,1329,1332,1000,2,2,C0004057,1,ab,"Nor did administration of SA, diflunisal or AS...",n
1,P3101,8369307,INHIBITS,VERB,890,899,rHF,aapp,785,788,...,912,919,888,1,15,C0242417,1,ab,A comparative study of recombinant L-cha...,n
2,P3102,3711333,INHIBITS,VERB,1527,1534,alkaloids,orch,1508,1517,...,1541,1550,1000,1,1,C0003805,1,ab,These findings suggest that some nicotinic alk...,y
3,P3103,11742534,INTERACTS_WITH,NOM,746,753,amino acids,aapp,703,714,...,741,745,694,0,4,C0169658|3716,1,ab,With a truncated chimaeric IL-5Rbeta-gp1...,y
4,P3104,244385,STIMULATES,ADJ,410,419,Neutral endopeptidase,aapp,374,401,...,480,491,1000,3,5,C0039815,1,ab,"Neutral endopeptidase, a zinc-dependent ...",n


In [223]:
df.columns

Index(['PREDICATION_ID', 'PMID', 'PREDICATE', 'INDICATOR_TYPE',
       'PREDICATE_START_INDEX', 'PREDICATE_END_INDEX', 'SUBJECT_TEXT',
       'SUBJECT_SEMTYPE', 'SUBJECT_START_INDEX', 'SUBJECT_END_INDEX',
       'SUBJECT_SCORE', 'SUBJECT_DIST', 'SUBJECT_MAXDIST', 'SUBJECT_CUI',
       'SUBJECT_NOVELTY', 'OBJECT_TEXT', 'OBJECT_SEMTYPE',
       'OBJECT_START_INDEX', 'OBJECT_END_INDEX', 'OBJECT_SCORE', 'OBJECT_DIST',
       'OBJECT_MAXDIST', 'OBJECT_CUI', 'OBJECT_NOVELTY', 'TYPE', 'SENTENCE',
       'LABEL'],
      dtype='object')

In [224]:
def pre_processing(example):
    sentence = example['SENTENCE']
    subject = example['SUBJECT_TEXT']
    object = example['OBJECT_TEXT']
    relation = example['PREDICATE']
    # text = f"{subject} [SEP] {relation} [SEP] {object} [SEP] {sentence}"
    text = f"{sentence} [SEP] {subject} , {relation} , {object}"
    return text

df['triple_with_sentence'] = df.apply(pre_processing,axis=1)
df.head()

,PREDICATION_ID,PMID,PREDICATE,INDICATOR_TYPE,PREDICATE_START_INDEX,PREDICATE_END_INDEX,SUBJECT_TEXT,SUBJECT_SEMTYPE,SUBJECT_START_INDEX,SUBJECT_END_INDEX,...,OBJECT_END_INDEX,OBJECT_SCORE,OBJECT_DIST,OBJECT_MAXDIST,OBJECT_CUI,OBJECT_NOVELTY,TYPE,SENTENCE,LABEL,triple_with_sentence
0,P3100,6499897,INTERACTS_WITH,NOM,1298,1304,SA,orch,1235,1237,...,1332,1000,2,2,C0004057,1,ab,"Nor did administration of SA, diflunisal or AS...",n,"Nor did administration of SA, diflunisal or AS..."
1,P3101,8369307,INHIBITS,VERB,890,899,rHF,aapp,785,788,...,919,888,1,15,C0242417,1,ab,A comparative study of recombinant L-cha...,n,A comparative study of recombinant L-cha...
2,P3102,3711333,INHIBITS,VERB,1527,1534,alkaloids,orch,1508,1517,...,1550,1000,1,1,C0003805,1,ab,These findings suggest that some nicotinic alk...,y,These findings suggest that some nicotinic alk...
3,P3103,11742534,INTERACTS_WITH,NOM,746,753,amino acids,aapp,703,714,...,745,694,0,4,C0169658|3716,1,ab,With a truncated chimaeric IL-5Rbeta-gp1...,y,With a truncated chimaeric IL-5Rbeta-gp1...
4,P3104,244385,STIMULATES,ADJ,410,419,Neutral endopeptidase,aapp,374,401,...,491,1000,3,5,C0039815,1,ab,"Neutral endopeptidase, a zinc-dependent ...",n,"Neutral endopeptidase, a zinc-dependent ..."


## Tokenizer

In [225]:

from transformers import AutoTokenizer

# Get model's tokenizer.
print('Loading tokenizer...')
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
# # default to left padding
# tokenizer.padding_side = "left"
# # Define PAD Token = EOS Token = 50256
# tokenizer.pad_token = tokenizer.eos_token
# tokenizer.add_special_tokens({'pad_token': '[PAD]'})

Loading tokenizer...


#### test tokenizer for a triple with corresponding sentence

In [226]:
example = df['triple_with_sentence'].iloc[0]
example

'Nor did administration of SA, diflunisal or ASA itself impair the       anti-aggregatory effect of a fresh test dose of ASA. [SEP] SA , INTERACTS_WITH , ASA'

In [227]:
tokenizer(example, return_tensors='pt', truncation=True)

{'input_ids': tensor([[21991,   750,  3662,   286, 14719,    11,   288,   361,    75,   403,
         28456,   393, 49599,  2346, 17253,   262,   220,   220,   220,   220,
           220,   220,  3098,    12,  9460,  2301,  2870,  1245,   286,   257,
          4713,  1332, 10742,   286, 49599,    13,   685,  5188,    47,    60,
         14719,   837, 23255,  2246,  4694,    62,    54, 10554,   837, 49599]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1]])}

In [228]:
def processing(example):
    res = tokenizer(example['triple_with_sentence'])
    # res['label'] = example['LABEL']
    res['label'] = 1 if example['LABEL']=='y' else 0
    return res
df['data'] = df.apply(processing, axis=1)

In [229]:
df['data'][0]

{'input_ids': [21991, 750, 3662, 286, 14719, 11, 288, 361, 75, 403, 28456, 393, 49599, 2346, 17253, 262, 220, 220, 220, 220, 220, 220, 3098, 12, 9460, 2301, 2870, 1245, 286, 257, 4713, 1332, 10742, 286, 49599, 13, 685, 5188, 47, 60, 14719, 837, 23255, 2246, 4694, 62, 54, 10554, 837, 49599], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'label': 0}

In [230]:
df.columns

Index(['PREDICATION_ID', 'PMID', 'PREDICATE', 'INDICATOR_TYPE',
       'PREDICATE_START_INDEX', 'PREDICATE_END_INDEX', 'SUBJECT_TEXT',
       'SUBJECT_SEMTYPE', 'SUBJECT_START_INDEX', 'SUBJECT_END_INDEX',
       'SUBJECT_SCORE', 'SUBJECT_DIST', 'SUBJECT_MAXDIST', 'SUBJECT_CUI',
       'SUBJECT_NOVELTY', 'OBJECT_TEXT', 'OBJECT_SEMTYPE',
       'OBJECT_START_INDEX', 'OBJECT_END_INDEX', 'OBJECT_SCORE', 'OBJECT_DIST',
       'OBJECT_MAXDIST', 'OBJECT_CUI', 'OBJECT_NOVELTY', 'TYPE', 'SENTENCE',
       'LABEL', 'triple_with_sentence', 'data'],
      dtype='object')

## split the data, training set 70%, validation set 15%, test set 15%

In [231]:
from sklearn.model_selection import train_test_split


train_data, test_data = train_test_split(df, test_size=0.3, random_state=42)
val_data, test_data = train_test_split(test_data, test_size=0.5, random_state=42)

In [232]:
print(len(train_data),len(val_data),len(test_data))

2100 450 450


In [233]:
train_data = train_data.reset_index()
val_data = val_data.reset_index()
test_data = test_data.reset_index()

In [234]:
from transformers import DataCollatorWithPadding
# import evaluate

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
# accuracy = evaluate.load('accuracy')

In [235]:
import numpy as np
from sklearn.metrics import precision_recall_fscore_support, accuracy_score


def compute_metrics(eval_pred):
	    predictions, labels = eval_pred
	    predictions = np.argmax(predictions, axis=1)
	    
	    # Calculate precision, recall, and F1 score
	    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='binary')
	    
	    return {
	        'accuracy': accuracy_score(labels, predictions),
	        'precision': precision,
	        'recall': recall,
	        'f1': f1
	    }

## BERT model

In [244]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from transformers import FalconForSequenceClassification

labels = ['n', 'y']
id2label = {i: label for i, label in enumerate(labels)}
label2id = {label: i for i, label in id2label.items()}

print('id2label:', id2label)
print('label2id:', label2id)

model = FalconForSequenceClassification.from_pretrained(MODEL_NAME)
# num_labels=len(labels), id2label=id2label, label2id=label2id

# Only train last classifier layer
# for param in model.base_model.parameters():
#     param.requires_grad = False
# # resize model embedding to match new tokenizer
# model.resize_token_embeddings(len(tokenizer))

# # fix model padding token id
# model.config.pad_token_id = model.config.eos_token_id

id2label: {0: 'n', 1: 'y'}
label2id: {'n': 0, 'y': 1}


Some weights of FalconForSequenceClassification were not initialized from the model checkpoint at Rocketknight1/falcon-rw-1b and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [245]:
model

FalconForSequenceClassification(
  (transformer): FalconModel(
    (word_embeddings): Embedding(50304, 2048)
    (h): ModuleList(
      (0-23): 24 x FalconDecoderLayer(
        (self_attention): FalconAttention(
          (query_key_value): FalconLinear(in_features=2048, out_features=6144, bias=True)
          (dense): FalconLinear(in_features=2048, out_features=2048, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (mlp): FalconMLP(
          (dense_h_to_4h): FalconLinear(in_features=2048, out_features=8192, bias=True)
          (act): GELU(approximate='none')
          (dense_4h_to_h): FalconLinear(in_features=8192, out_features=2048, bias=True)
        )
        (input_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
      )
    )
    (ln_f): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
  )
  (score): Linear(in_features=2048,

In [239]:
# # Freeze all layers except the last one
# for param in model.base_model.parameters():
#     param.requires_grad = False

# # for param in model.bert.pooler.dense.parameters():
# #     param.requires_grad = True

# # # Unfreeze the last three layers
# # for param in model.transformer.ln_f.parameters():
# #     param.requires_grad = True

# for param in model.parameters():
#     print(param.requires_grad)

##  Training

In [240]:
training_args = TrainingArguments(
    output_dir='my_best_model',
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=10,
    weight_decay=0.01,
    evaluation_strategy='epoch',
    save_strategy='epoch',
    load_best_model_at_end=True
)

In [241]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data['data'],
    eval_dataset=val_data['data'],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)
trainer.train()

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [242]:
trainer.evaluate(test_data['data'])

AttributeError: 'AcceleratorState' object has no attribute 'distributed_type'

In [ ]:
import torch
from transformers import AutoTokenizer, FalconForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("Rocketknight1/falcon-rw-1b")
model = FalconForSequenceClassification.from_pretrained("Rocketknight1/falcon-rw-1b")

inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")
print(inputs)

with torch.no_grad():
    logits = model(**inputs).logits

predicted_class_id = logits.argmax().item()
print(predicted_class_id)

In [ ]:
model